In [15]:
#Importing libraries
import pandas as pd
import numpy as np
import math

In [16]:
#Loading the shots data
shots = pd.read_csv("/Users/mikesepe/Downloads/shots_data.csv")

shots.head()

,team,x,y,fgmade
0,Team A,-23.1,3.5,0
1,Team A,0.0,25.1,1
2,Team A,0.5,1.0,1
3,Team A,-5.6,5.2,0
4,Team A,4.0,14.5,1


In [43]:
#creating dummy variable for non corner threes
shots['NC3'] = 0
shots['NC3_made'] = 0

# Selecting only shots withing the geometric criteria
for i in range(len(shots)):
    if (np.abs(shots['x'][i]) > 22 and shots['y'][i] > 7.8) or np.abs(shots['x'][i]) < 22 and shots['y'][i] > 0.25*math.sqrt(9025 - 16*shots['x'][i]):
        shots['NC3'][i] = 1
        if shots['fgmade'][i] == 1:
            shots['NC3_made'][i] = 1


/Users/mikesepe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/mikesepe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
# Creating a dummy for corner threes
shots['C3'] = 0
shots['C3_made'] = 0

# Selecting only shots withing the geometric criteria
for i in range(len(shots)):
    if np.abs(shots['x'][i]) > 22 and shots['y'][i] < 7:
        shots['C3'][i] = 1
        if shots['fgmade'][i] == 1:
            shots['C3_made'][i] = 1

/Users/mikesepe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/mikesepe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [45]:
# Creating a dummy for two point shots
shots['2PT'] = 0
shots['2PT_made'] = 0

# Selecting only shots withing the geometric criteria
for i in range(len(shots)):
    if np.abs(shots['x'][i]) <= 22 and shots['y'][i] <= 0.25*math.sqrt(9025 - 16*shots['x'][i]):
        shots['2PT'][i] = 1
        if shots['fgmade'][i] == 1:
            shots['2PT_made'][i] = 1

/Users/mikesepe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/mikesepe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [58]:
# Showing the final dataframe with the finished dummies
shots.head()

,team,x,y,fgmade,NC3,C3,2PT,2PT_made,C3_made,NC3_made
0,Team A,-23.1,3.5,0,0,1,0,0,0,0
1,Team A,0.0,25.1,1,1,0,0,0,0,1
2,Team A,0.5,1.0,1,0,0,1,1,0,0
3,Team A,-5.6,5.2,0,0,0,1,0,0,0
4,Team A,4.0,14.5,1,0,0,1,1,0,0


In [64]:
# Dropping the cartesian coordinates now that I dont need them
grouped_team = pd.DataFrame(shots.groupby('team').sum().drop(['x','y'], axis = 1))

grouped_team.head()

,fgmade,NC3,C3,2PT,2PT_made,C3_made,NC3_made
team,,,,,,,
Team A,105,45,19,215,80,8,17
Team B,115,42,20,217,95,5,15


In [65]:
# Total shots by adding up all of the shots taken
grouped_team['total'] = grouped_team['2PT'] + grouped_team['C3'] + grouped_team['NC3']

In [66]:
# Creating effective field goal percentages by making proportions
grouped_team['eFG%_2PT'] = grouped_team['2PT_made']/grouped_team['2PT']
grouped_team['eFG%_C3'] = grouped_team['C3_made']/grouped_team['C3']
grouped_team['eFG%_NC3'] = grouped_team['NC3_made']/grouped_team['NC3']

In [67]:
# Creating the distribution of shots by region on the floor determined earlier by the cartesian coordinates
# and the if statements above
grouped_team['2PT_distrib'] = grouped_team['2PT']/grouped_team['total']
grouped_team['C3_distrib'] = grouped_team['C3']/grouped_team['total']
grouped_team['NC3_distrib'] = grouped_team['NC3']/grouped_team['total']

In [68]:
# Displaying the answers that I have generated
grouped_team.head()

,fgmade,NC3,C3,2PT,2PT_made,C3_made,NC3_made,total,eFG%_2PT,eFG%_C3,eFG%_NC3,2PT_distrib,C3_distrib,NC3_distrib
team,,,,,,,,,,,,,,
Team A,105,45,19,215,80,8,17,279,0.372093,0.421053,0.377778,0.770609,0.068100,0.161290
Team B,115,42,20,217,95,5,15,279,0.437788,0.250000,0.357143,0.777778,0.071685,0.150538


In [72]:
# Outputting the answers to a xlsx file
grouped_team.to_excel('/Users/mikesepe/Downloads/Mike_sepe_OKC_challenge_answer.xlsx')

# Thanks for viewing!